## Import Tools

In [1]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io 
import random
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import seaborn as sn
import pandas as pd
os.environ['KMP_DUPLICATE_LIB_OK']='True' 

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

## Dataset Processing 

### Read in the original dataset 

In [2]:
train_dl_origin = torch.load('Dataset/train_dl.pt')
valid_dl_origin = torch.load('Dataset/valid_dl.pt')

train_CSI = train_dl_origin.dataset[:][0]
train_label = train_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

valid_CSI = valid_dl_origin.dataset[:][0]
valid_label = valid_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

In [3]:
print(train_label)

tensor([1, 1, 0,  ..., 0, 0, 1])


In [4]:
print(train_label.shape)

torch.Size([15000])


### CSI Processing: Take Modulus of complex matrices

In [5]:
train_CSI_modulus = torch.abs(train_CSI)
valid_CSI_modulus = torch.abs(valid_CSI)

In [6]:
print(train_CSI_modulus)

tensor([[[[100.6578, 124.7878, 106.1179,  ..., 304.7704, 299.6064, 324.3594],
          [132.8157, 106.6771,  91.2688,  ..., 269.1561, 323.5568, 299.9617],
          [129.1395, 148.4756, 170.0735,  ..., 399.8112, 407.4420, 402.0112],
          [ 74.0000,  71.4493,  59.3633,  ..., 134.0149, 129.6919, 124.0363]]],


        [[[177.0198, 170.4963, 169.1065,  ...,  46.6154,  37.6431,  64.4981],
          [143.6802, 143.0874,  88.0909,  ...,  44.0454,  22.2036,  27.6586],
          [ 97.8008,  80.7527,  71.7008,  ...,  32.2025,  22.4722,  39.3573],
          [ 39.8121,  45.7930,  31.6228,  ...,  16.5529,   8.0623,  25.6125]]],


        [[[411.3940, 421.5412, 380.1276,  ..., 509.8431, 550.0582, 539.8120],
          [366.8079, 387.3629, 353.0340,  ..., 596.1241, 619.6975, 605.5353],
          [574.8991, 593.8560, 612.0008,  ..., 928.9521, 923.3618, 914.0552],
          [289.8362, 287.2368, 281.0427,  ..., 354.9113, 339.0634, 333.9461]]],


        ...,


        [[[296.5889, 288.2672, 292.76

In [7]:
print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])


###  CSI Processing: Normalize to [0,1]

In [8]:
# Min-Max Scaling
min_value = torch.min(train_CSI_modulus)
max_value = torch.max(train_CSI_modulus)

normalized_train_CSI_modulus = (train_CSI_modulus - min_value) / (max_value - min_value)
normalized_valid_CSI_modulus = (valid_CSI_modulus - min_value) / (max_value - min_value)


### ML Classifcation w/ KNN

In [9]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [10]:
# Convert 4d to 2d
train_data_2d = normalized_train_CSI_modulus.view(normalized_train_CSI_modulus.size(0), -1)  # Reshape to (15000, 4 * 1632)
valid_data_2d = normalized_valid_CSI_modulus.view(normalized_valid_CSI_modulus.size(0), -1)  # Reshape to (15000, 4 * 1632)

In [11]:
# Shuffle and split 
x, y = np.array(train_data_2d), np.array(train_label)
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42) 


In [12]:
sq = int(np.sqrt(15000))
knn = KNeighborsClassifier(n_neighbors=35)
n_folds = 5
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

train_scores = []
valid_scores = []
for train_index, valid_index in kf.split(X_train, y_train):
    X_train_fold, X_valid_fold = X_train[train_index], X_train[valid_index]
    y_train_fold, y_valid_fold = y_train[train_index], y_train[valid_index]

    knn.fit(X_train_fold, y_train_fold)

    train_score = knn.score(X_train_fold, y_train_fold)
    valid_score = knn.score(X_valid_fold, y_valid_fold)

    train_scores.append(train_score)
    valid_scores.append(valid_score)

mean_train_score = np.mean(train_scores)
mean_valid_score = np.mean(valid_scores)

print('Mean accuracy of KNN classifier on training set: {:.2f}'.format(mean_train_score))
print('Mean accuracy of KNN classifier on validation set: {:.2f}'.format(mean_valid_score))


C:\Users\Joey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Joey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavio

Mean accuracy of KNN classifier on training set: 0.96
Mean accuracy of KNN classifier on validation set: 0.94


C:\Users\Joey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


### ANN approach (MLP)

- Instantiate a Neural Network Model

In [30]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(4*1632, 512)  # input layer (4*1632 nodes) -> hidden layer (50 nodes)
        self.layer2 = nn.Linear(512, 256)  # hidden layer (50 nodes) -> hidden layer (50 nodes)
        self.layer3 = nn.Linear(256, 128)  # hidden layer (50 nodes) -> hidden layer (100 nodes)
        self.layer4 = nn.Linear(128, 1)  # hidden layer (100 nodes) -> output layer (1 nodes)

    def forward(self, x):
        return self.layer4(F.relu(self.layer3(F.relu(self.layer2(F.relu(self.layer1(x)))))))

net = Net()
print(net)

Net(
  (layer1): Linear(in_features=6528, out_features=512, bias=True)
  (layer2): Linear(in_features=512, out_features=256, bias=True)
  (layer3): Linear(in_features=256, out_features=128, bias=True)
  (layer4): Linear(in_features=128, out_features=1, bias=True)
)


- Add a loss function and an optimizer

In [38]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()  # define the loss function
optimizer = optim.SGD(net.parameters(), lr=0.015, momentum=0.9)  # define the optimizer

- Train the neural network

In [39]:
train_data_2d = torch.tensor(train_data_2d, dtype=torch.float32)
train_label = torch.tensor(train_label, dtype=torch.float32)
train_label = train_label.view(-1, 1)

num_epochs = 25
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = net(train_data_2d)
    loss = criterion(output, train_label)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

C:\Users\Joey\AppData\Local\Temp\ipykernel_10040\2307469813.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data_2d = torch.tensor(train_data_2d, dtype=torch.float32)
C:\Users\Joey\AppData\Local\Temp\ipykernel_10040\2307469813.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_label = torch.tensor(train_label, dtype=torch.float32)


Epoch [1/25], Loss: 0.4443829357624054
Epoch [2/25], Loss: 0.4438953995704651
Epoch [3/25], Loss: 0.44299155473709106
Epoch [4/25], Loss: 0.44175466895103455
Epoch [5/25], Loss: 0.44027525186538696
Epoch [6/25], Loss: 0.43864747881889343
Epoch [7/25], Loss: 0.4369651675224304
Epoch [8/25], Loss: 0.4353173077106476
Epoch [9/25], Loss: 0.4337822496891022
Epoch [10/25], Loss: 0.43242383003234863
Epoch [11/25], Loss: 0.43128907680511475
Epoch [12/25], Loss: 0.43040451407432556
Epoch [13/25], Loss: 0.42977529764175415
Epoch [14/25], Loss: 0.42938661575317383
Epoch [15/25], Loss: 0.42920684814453125
Epoch [16/25], Loss: 0.42919111251831055
Epoch [17/25], Loss: 0.42928802967071533
Epoch [18/25], Loss: 0.42944544553756714
Epoch [19/25], Loss: 0.429614782333374
Epoch [20/25], Loss: 0.429755836725235
Epoch [21/25], Loss: 0.4298388957977295
Epoch [22/25], Loss: 0.4298464059829712
Epoch [23/25], Loss: 0.42977216839790344
Epoch [24/25], Loss: 0.4296202063560486
Epoch [25/25], Loss: 0.42940333485603

- Test the Neural Network

In [40]:
def test_accuracy(model, test_data, test_labels):
    model.eval()

    with torch.no_grad():
        test_predictions = model(test_data)

    rounded_predictions = torch.round(torch.sigmoid(test_predictions))
    accuracy = accuracy_score(test_labels, rounded_predictions)
    return accuracy

accuracy = test_accuracy(net, valid_data_2d, valid_label)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 81.28%
